In [55]:
import sys
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import requests as rq

In [56]:
#returns data from from HTML table cell
def date_time(table_cells):
    return[data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    out=''.join([booster_version for i, booster_version in enumerate(table_cells.strings)if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    out=[i for i in table_cells.strings][0]
    return out

def get_mass(table_cells):
    mass = unicodedata.normalize('NFKD', table_cells.text).strip()
    if mass:
        mass.find('kg')
        new_mass = mass[0:mass.find('kg')+2]
    else:
        new_mass = 0
    return new_mass

def extract_column_from_header(row):
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    column_name = ' '.join(row.contents)

    if not(column_name.strip().isdigit()):
        column_name = column_name.strip()
        return column_name


In [57]:
#html page used for Falcon 9 launch data
static_url = 'https://en.wikipedia.org/w/index.php?title-List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922'

In [58]:
#store the request as a response
response_content = rq.get(static_url).text

In [59]:
#create an object from the response
soup = BeautifulSoup(response_content, 'html5lib')

In [60]:
#print the title to confirm creation
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

In [61]:
#create list to store names
html_tables = soup.find_all('table')

In [62]:
first_launch_table = html_tables[3]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a><sup class="reference" id="cite_ref-booster_11-1"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-1"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">8
</th>
<td>6

In [63]:
column_names = []
labels = first_launch_table.find_all('th')
for label in labels:
    name = extract_column_from_header(label)
    if name != None:
        if len(name) > 0:
            column_names.append(name)

In [64]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [65]:
launch_dict= dict.fromkeys(column_names)

del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [66]:
extracted_row = 0
for table_number, table in enumerate(soup.find_all('table', 'wikitable plainrowheaders collapsible')):
    for rows in table.find_all('tr'):
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False 
        row = rows.find_all('td')
        if flag:
            extracted_row += 1
            
            launch_dict['Flight No.'].append(flight_number)
            print(flight_number)
            datatimelist = date_time(row[0])

            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            print(date)

            time = datatimelist[1]
            launch_dict['Time'].append(time) 
            print(time)

            bv = booster_version(row[1])
            if not (bv):
                bv = row[1].a.string
            launch_dict['Version Booster']. append(bv)
            print(bv)

            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            print(launch_site)

            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            print(payload)

            payload_mass =  get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            print(payload_mass)

            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            print(orbit)

            customer = row[6].text.strip()
            launch_dict['Customer'].append(customer)
            print(customer)

            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'] .append(launch_outcome)
            print(launch_outcome)

            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)
            print(booster_landing)

1
4 June 2010
18:45
F9 v1.0B0003.1
CCAFS
Dragon Spacecraft Qualification Unit
0
LEO
SpaceX
Success

Failure
2
8 December 2010
15:43
F9 v1.0B0004.1
CCAFS
Dragon
0
LEO
.mw-parser-output .plainlist ol,.mw-parser-output .plainlist ul{line-height:inherit;list-style:none;margin:0;padding:0}.mw-parser-output .plainlist ol li,.mw-parser-output .plainlist ul li{margin-bottom:0}
NASA (COTS)
NRO
Success
Failure
3
22 May 2012
07:44
F9 v1.0B0005.1
CCAFS
Dragon
525 kg
LEO
NASA (COTS)
Success
No attempt

4
8 October 2012
00:35
F9 v1.0B0006.1
CCAFS
SpaceX CRS-1
4,700 kg
LEO
NASA (CRS)
Success

No attempt
5
1 March 2013
15:10
F9 v1.0B0007.1
CCAFS
SpaceX CRS-2
4,877 kg
LEO
NASA (CRS)
Success

No attempt

6
29 September 2013
16:00
F9 v1.1B1003
VAFB
CASSIOPE
500 kg
Polar orbit
MDA
Success
Uncontrolled
7
3 December 2013
22:41
F9 v1.1
CCAFS
SES-8
3,170 kg
GTO
SES
Success
No attempt
8
6 January 2014
22:06
F9 v1.1
CCAFS
Thaicom 6
3,325 kg
GTO
Thaicom
Success
No attempt
9
18 April 2014
19:25
F9 v1.1
Cape Canav

In [69]:
df = pd.DataFrame(launch_dict)

In [68]:
df.describe()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
count,121,121,121,121,121,121,121,121,121,121,121
unique,121,5,82,84,8,54,3,52,8,121,116
top,1,CCAFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,F9 B5,Success,4 June 2010,04:45
freq,1,40,29,24,67,26,88,26,80,1,3


In [70]:
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,".mw-parser-output .plainlist ol,.mw-parser-out...",Success,F9 v1.0B0004.1,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA (COTS),Success,F9 v1.0B0005.1,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA (CRS),Success\n,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA (CRS),Success\n,F9 v1.0B0007.1,No attempt\n,1 March 2013,15:10
